`AdaLoRA` is a method for optimizing the number of trainable parameters to assign to weight matrices and layers, unlike LoRA, which distributes parameters evenly across all modules. More parameters are budgeted for important weight matrices and layers while less important ones receive fewer parameters.

`LoHa` (Low-Rank Hadamard Product), is similar to LoRA except it approximates the large weight matrix with more low-rank matrices and combines them with the Hadamard product. This method is even more parameter-efficient than LoRA and achieves comparable performance.

`LoKr` (Low-Rank Kronecker Product), is a LoRA-variant method that approximates the large weight matrix with two low-rank matrices and combines them with the Kronecker product. LoKr also provides an optional third low-rank matrix to provide better control during fine-tuning.


In [76]:
import torch

import sys
sys.path.append('..')
import warnings
warnings.filterwarnings('ignore')

from Data import load_and_prepare_dataset_for_layoutLM
from transformers import LayoutLMForTokenClassification, get_linear_schedule_with_warmup
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup
from Training import fine_tune_model, evaluate_model, apply_peft

from sklearn.model_selection import ParameterGrid
from itertools import combinations, chain
import pickle

from peft import get_peft_config, get_peft_model, LoraConfig, LoHaConfig, LoKrConfig, AdaLoraConfig, TaskType 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
id2label, label2id, train_dataloader, val_dataloader, test_dataloader = load_and_prepare_dataset_for_layoutLM(batch_size=2)

Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [78]:
device = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu')) # use 'cuda' instead of 'mps' if running on a GPU machine that is not provided by MacOS

In [71]:
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [82]:
NUM_EXPERIMENTS = 20
results_list = []

param_grid = {
    'peft_method': [LoraConfig, LoHaConfig, LoKrConfig, AdaLoraConfig],
    'r': [16], #[2**i for i in range(1, 7)],  # values from 2 to 64, all power of 2
    'target_modules': [['key', 'value', 'query', 'attention.output.dense']]#list(chain(*[combinations(['query', 'key', 'value', 'attention.output.dense'], i) for i in range(1, 6)]))
}


for _ in range(NUM_EXPERIMENTS):
    results_dict = {}

    for params in ParameterGrid(param_grid):
            
        training_losses_list, val_losses_list, training_f1s_list, val_f1s_list, training_accs_list, val_accs_list = [], [], [], [], [], []

        print("\n")
        print("====================================================================================================")
        print(f"Running experiment with parameters: {params['peft_method'].__name__}, r={params['r']}, target_modules={params['target_modules']}")
        print("====================================================================================================")

        model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=len(id2label),id2label=id2label, label2id=label2id)
        model, trainable_parameters = apply_peft(model, **params)
        model.to(device)
        optimizer = AdamW(model.parameters(), lr=5e-4)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=2, num_training_steps=30)


        training_losses, val_losses, training_f1s, val_f1s, training_accs, val_accs = fine_tune_model(
                model=model,
                device=device,
                optimizer=optimizer,
                scheduler=scheduler,
                train_dataloader=train_dataloader,
                val_dataloader=val_dataloader,
                id2label=id2label,
                num_epoch_max=20,
                early_stopping=False,
                test_frequency=1,
                model_folder='Lora_Fine_Tuning',
                model_name=f'Lora_{params["peft_method"]}',
            )
        
        training_losses_list.append(training_losses)
        val_losses_list.append(val_losses)
        training_f1s_list.append(training_f1s)
        val_f1s_list.append(val_f1s)
        training_accs_list.append(training_accs)
        val_accs_list.append(val_accs)
        
        results = evaluate_model(
            model=model,
            device=device,
            test_dataloader=test_dataloader,
            original_id2label=id2label
        )

        results['train_losses'] = training_losses
        results['val_losses'] = val_losses
        results['training_f1s'] = training_f1s
        results['val_f1s'] = val_f1s
        results['training_accs'] = training_accs
        results['val_accs'] = val_accs
        results['trainable_parameters'] = trainable_parameters

        results_dict[params['peft_method'].__name__] = results
        print('\n')

    results_list.append(results_dict)
    with open('results_list_lora_method_study_2.pkl', 'wb') as f:
        pickle.dump(results_list, f)
   





Running experiment with parameters: LoraConfig, r=16, target_modules=['key', 'value', 'query', 'attention.output.dense']


Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,185,031 || all params: 113,818,382 || trainable%: 1.041159590548388
None


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Time elapsed,▃▁▇██
it_s,▆█▂▁▁
Time elapsed,20.55946
it_s,0.09728


...

Test: 100%|██████████| 23/23 [00:03<00:00,  5.82it/s]


Epoch Test loss: 0.3601008290829866, Test F1 Score: 0.7849894696534558, Test Accuracy: 0.8224674022066198
Epoch: 7/20


Training:  52%|█████▏    | 27/52 [00:13<00:12,  2.03it/s]


KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.
